# Studi Kasus Beer Production

## PROFILE KELOMPOK


NPM KETUA : 200411100153

NAMA KETUA : FIKRI AINUN NAJIB

NPM ANGGOTA : 200411100166

NAMA ANGGOTA : GHANIY ALBAR RASYIDI KUSUMA

GITHUB : https://github.com/LALA09-erha/kolaborasipro

DATASET : https://www.kaggle.com/datasets/shenba/time-series-datasets?select=monthly-beer-production-in-austr.csv

MODEL : KNN REGRESI , GAUSSIAN REGRESI , bagging Regressor > SVR , random forest Regressor , stacking Regressor > [RidgeCV ,LinearSVR], DummyRegressor, LinearRegression, DecisionTreeRegressor

KELAS PSD C

Inisialisasi Libary dan beberapa Model

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from numpy import array
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor

Pengambilan data

In [30]:
data = pd.read_csv("../Data/monthly-beer-production-in-austr.csv")

# add new data input to the dataframe
# data = data.append({'Month': '1995-09', 'Monthly beer production': 0}, ignore_index=True)
data = data ['Monthly beer production']


Split Data

In [31]:
data_training,data_test = train_test_split(data, train_size=0.8, test_size=0.2, shuffle=False, random_state=42)

In [32]:
data_training_new = pd.DataFrame(data_training)
data_test_new = pd.DataFrame(data_test)

Preprosessing Data

In [33]:
sc = MinMaxScaler()
train_scaled = sc.fit_transform(data_training_new)
test_scaled = sc.transform(data_test_new)

fungsi Cross Validation

In [34]:
# Cross Validation
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  
  return array(X), array(y)
# define univariate time series

Eksekusi Model

In [35]:
# inisialisasi variabel untuk menampung hasil akurasi dan n_steps
n_steps = range(1,7)
acc_fix = {}

for n in n_steps:
    # Proses Pemanggilan Cross Validation 
    X_train, y_train = split_sequence(train_scaled, n) 
    X_test, y_test = split_sequence(test_scaled, n) 

    # membuat kolom otomatis
    colom = []
    for c in range(n):
        if(c==0):
            colom.insert(0,"Xt")
        else:
            colom.insert(0,f"Xt-{c}")

    # # Masukkan ke dataframe hasil dari split  squens
    # UNTUK MEMISAHKAN X TRAIN, YTRAIN dan TEST
    newX_train = pd.DataFrame(np.reshape(X_train, (len(X_train) , n)),columns=colom)
    newY_train = pd.DataFrame(y_train, columns=['Output'])
    new_data_train = pd.concat([newX_train,newY_train],axis=1)

    newX_test = pd.DataFrame(np.reshape(X_test, (len(X_test) , n)),columns=colom)
    newY_test = pd.DataFrame(y_test, columns=['Output'])
    new_data_test = pd.concat([newX_test,newY_test],axis=1)


    # EKSEKUSI KE model

    # predict knn
    k_nei = range(1,30)
    acc = []
    for k in k_nei:
        knn = KNeighborsRegressor(n_neighbors = k)
        knn.fit(newX_train,newY_train)
        y_pred = knn.predict(newX_test)
        y_pred_shape = y_pred.reshape(-1,1)
        y_pred_inverse = sc.inverse_transform(y_pred_shape)
        newY_test_shape = y_test.reshape(-1,1)
        newY_test_inverse = sc.inverse_transform(newY_test_shape)
        mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
        acc.append(mape)
    
    # masukkan ke acc-fix 
    acc_fix[f"n-{n} knn"] = [min(acc) , acc.index(min(acc))+1]
        
    # predict gaussian regressor
    kernel = DotProduct() + WhiteKernel()
    gpr = GaussianProcessRegressor(kernel=kernel,random_state=0).fit(newX_train, newY_train)
    y_pred = gpr.predict(newX_test, return_std=True)[0]
    y_pred_shape = y_pred.reshape(-1,1)
    y_pred_inverse = sc.inverse_transform(y_pred_shape)
    newY_test_shape = y_test.reshape(-1,1)
    newY_test_inverse = sc.inverse_transform(newY_test_shape)
    # print(y_pred_inverse)
    mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)

    acc_fix[f"n-{n} gpr"] = [mape]

    # Predict SVR dengan Bagging Regressor
    n_estimators = range(1,30)
    acc_svr = []
    for e in n_estimators:
        regr = BaggingRegressor(SVR(), n_estimators=e, random_state=0).fit(newX_train, newY_train.values.ravel())
        y_pred = regr.predict(newX_test)
        y_pred_shape = y_pred.reshape(-1,1)
        y_pred_inverse = sc.inverse_transform(y_pred_shape)
        newY_test_shape = y_test.reshape(-1,1)
        newY_test_inverse = sc.inverse_transform(newY_test_shape)
        mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
        acc_svr.append(mape)
    
    # masukkan ke acc-fix
    acc_fix[f"n-{n} bagsvr"] = [min(acc_svr) , acc_svr.index(min(acc_svr))+1]

    # Predict dengan from RandomForestRegressor
    maxd = range(1,10)
    acc_rf = []
    for d in maxd:
        regr = RandomForestRegressor(max_depth=d, random_state=0).fit(newX_train, newY_train.values.ravel())
        y_pred = regr.predict(newX_test)
        y_pred_shape = y_pred.reshape(-1,1)
        y_pred_inverse = sc.inverse_transform(y_pred_shape)
        newY_test_shape = y_test.reshape(-1,1)
        newY_test_inverse = sc.inverse_transform(newY_test_shape)
        mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
        acc_rf.append(mape)
    
    # masukkan ke acc-fix
    acc_fix[f"n-{n} rf"] = [min(acc_rf) , acc_rf.index(min(acc_rf))+1]

    # Prdict dengan LinearSVR , Ridge pada stacking regressors
    estimators = [('lr', RidgeCV()),
                ('svr', LinearSVR(random_state=42, max_iter=100000))]
    acc_stac = []
    for e in n_estimators:
        reg = StackingRegressor(
            estimators,
            final_estimator=RandomForestRegressor(n_estimators=e,
                                                random_state=42))
        reg.fit(newX_train, newY_train.values.ravel())
        y_pred = reg.predict(newX_test)
        y_pred_shape = y_pred.reshape(-1,1)
        y_pred_inverse = sc.inverse_transform(y_pred_shape)
        newY_test_shape = y_test.reshape(-1,1)
        newY_test_inverse = sc.inverse_transform(newY_test_shape)
        mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
        acc_stac.append(mape)
    
    # masukkan ke acc-fix
    acc_fix[f"n-{n} stac"] = [min(acc_stac) , acc_stac.index(min(acc_stac))+1]

     
    #Predict dengan Decission Tree Regresor
    dt_max_depth = range(1, 10)
    acc_dt = []
    for depth in dt_max_depth:
        dt = DecisionTreeRegressor(max_depth=depth)
        dt.fit(newX_train, newY_train)
        y_pred_dt = dt.predict(newX_test)
        y_pred_dt_shape = y_pred_dt.reshape(-1, 1)
        y_pred_dt_inverse = sc.inverse_transform(y_pred_dt_shape)
        mape = mean_absolute_percentage_error(y_pred_dt_inverse, newY_test_inverse)
        acc_dt.append(mape)
        
    # masukkan ke acc-fix    
    acc_fix[f"n-{n} dt"] = [min(acc_dt) , acc_dt.index(min(acc_dt))+1]
    
    #Linier Regressor
    acc_lr =[]
    lr = LinearRegression()
    lr.fit(newX_train, newY_train)
    y_pred_lr = lr.predict(newX_test)
    y_pred_lr_shape = y_pred_lr.reshape(-1, 1)
    y_pred_lr_inverse = sc.inverse_transform(y_pred_lr_shape)
    mape_lr = mean_absolute_percentage_error(y_pred_lr_inverse, newY_test_inverse)
    acc_lr.append(mape)
    
    # masukkan ke acc-fix    
    acc_fix[f"n-{n} lr"] = [min(acc_lr) , acc_lr.index(min(acc_lr))+1]
    
    #Dummy Regressor
    acc_dr = []
    dummy = DummyRegressor()
    dummy.fit(newX_train, newY_train)
    y_pred_dummy = dummy.predict(newX_test)
    y_pred_dummy_shape = y_pred_dummy.reshape(-1, 1)
    y_pred_dummy_inverse = sc.inverse_transform(y_pred_dummy_shape)
    mape = mean_absolute_percentage_error(y_pred_dummy_inverse, newY_test_inverse)
    acc_dr.append(mape)
    
    
    # masukkan ke acc-fix    
    acc_fix[f"n-{n} dr"] = [min(acc_dr) , acc_dr.index(min(acc_dr))+1]
 

print(acc_fix)


{'n-1 knn': [0.10138941716331257, 29], 'n-1 gpr': [0.10682291733064105], 'n-1 bagsvr': [0.09949710233664405, 1], 'n-1 rf': [0.10225707462838196, 2], 'n-1 stac': [0.136058553083642, 6], 'n-1 dt': [0.1030645637997417, 3], 'n-1 lr': [0.12845760567583306, 1], 'n-1 dr': [0.18615750869387102, 1], 'n-2 knn': [0.10028274680900791, 17], 'n-2 gpr': [0.10906140141958981], 'n-2 bagsvr': [0.10173422756521319, 1], 'n-2 rf': [0.10119012776685196, 3], 'n-2 stac': [0.11035878149083046, 25], 'n-2 dt': [0.10552479496483443, 3], 'n-2 lr': [0.12849681048473338, 1], 'n-2 dr': [0.18590076516757667, 1], 'n-3 knn': [0.09585157865328107, 7], 'n-3 gpr': [0.10781311991234968], 'n-3 bagsvr': [0.09787453007520233, 29], 'n-3 rf': [0.09448243357134146, 5], 'n-3 stac': [0.1119226357705636, 15], 'n-3 dt': [0.10421884449670733, 4], 'n-3 lr': [0.1293403107566488, 1], 'n-3 dr': [0.18392753808599016, 1], 'n-4 knn': [0.08200835382664716, 5], 'n-4 gpr': [0.10648093277107598], 'n-4 bagsvr': [0.08687363259415466, 29], 'n-4 rf'

### Hasil Berbagai Model

Menentukan MAPE terendah dari Berbagai Model Diatas , mengambil 3 hasil terendah


In [42]:
dict_acc = {}
for m in acc_fix:
    dict_acc[m] = acc_fix[m]
# # mengambil 3 nilai terendah dari dict_acc
sorted_dict_acc = sorted(dict_acc.items(), key=lambda x: x[1])[:3]
print(sorted_dict_acc)


[('n-6 knn', [0.0707448021927608, 4]), ('n-6 rf', [0.07279313958249832, 8]), ('n-6 bagsvr', [0.07518563792189382, 2])]


- Yang terbaik adalah model dengan MAPE terendah Knn Regresi dengan k = 4 dan n = 6 
- Yang terbaik adalah model dengan MAPE terendah RandomForestRegressor  dengan d = 8 dan n = 6 
- Yang terbaik adalah model dengan MAPE terendah  SVR dengan Bagging Regressor  dengan e = 2 dan n = 6 